# 9.3 Agent 框架实战

本笔记本演示如何使用 OpenAI Agents SDK 构建 Agent。

**预估成本**: ~$0.05

```bash
pip install openai requests
```

In [ ]:
!pip install openai requests -q

In [ ]:
import os
from openai import OpenAI

client = OpenAI()
print('✓ OpenAI client initialized')

## Demo 1: 创建第一个 Agent

In [ ]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'calculator',
            'description': 'Calculate a math expression',
            'parameters': {
                'type': 'object',
                'properties': {'expression': {'type': 'string'}},
                'required': ['expression'],
            },
        },
    },
]

agent = client.beta.assistants.create(
    name='Math Assistant',
    instructions='You are a helpful math assistant. Use calculator when needed.',
    model='gpt-4o-mini',
    tools=tools,
)

print(f'Agent created: {agent.id}')

## Demo 2: 运行 Agent

In [ ]:
def run_agent(agent_id, message):
    thread = client.beta.threads.create()
    
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role='user',
        content=message,
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=agent_id,
    )
    
    # 等待完成（简化版）
    import time
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        
        if run_status.status == 'completed':
            break
        elif run_status.status == 'requires_action':
            # 处理工具调用
            pass
        
        time.sleep(1)
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages.data[0].content[0].text.value

result = run_agent(agent.id, 'What is 25 * 4?')
print(result)

## 总结

- ✓ 用框架从 150 行代码降到 15 行
- ✓ 自动处理循环和错误
- ✓ 原生支持多轮对话

下一节学习 Computer Use！